In [1]:
# download nltk stopwords
import nltk
from nltk.corpus import stopwords
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
from time import time
from pathlib import Path
import pickle
from google.cloud import storage
from collections import defaultdict
from contextlib import closing
import math
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Install a particular version of `google-cloud-storage` because (oddly enough)
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [3]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install pyngrok

In [5]:
# TODO: sign up for an ngrok account
# then put your ngrok token below and execute
from pyngrok import ngrok
ngrok.set_auth_token("2cXd1YwtWlYn3kGKfj97UtzuCER_42NZiY3AoMj93W18LwJWp")

# Run the app

In [6]:
# you need to upload your implementation of search_app.py
import search_frontend as se
import sim_calc
import inverted_index_title_gcp
import inverted_index_body_gcp_training

In [7]:
# uncomment the code below and execute to reload the module when you make
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(se)
importlib.reload(sim_calc)
importlib.reload(inverted_index_title_gcp)
importlib.reload(inverted_index_body_gcp_training)

<module 'inverted_index_body_gcp_training' from '/content/inverted_index_body_gcp_training.py'>

In [ ]:
public_url = ngrok.connect("5000", bind_tls=False).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
# Update any base URLs to use the public ngrok URL
se.app.config["BASE_URL"] = public_url
se.app.run()

 * ngrok tunnel "http://d116-34-80-115-88.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app 'search_frontend'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Feb/2024 00:52:56] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Feb/2024 00:52:57] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Feb/2024 00:53:03] "GET /search HTTP/1.1" 200 -


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [9]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [10]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [22]:
def precision_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(predicted_list) == 0:
        return 0.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(predicted_list), 3)
def recall_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(true_set) < 1:
        return 1.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(true_set), 3)
def f1_at_k(true_list, predicted_list, k):
    p = precision_at_k(true_list, predicted_list, k)
    r = recall_at_k(true_list, predicted_list, k)
    if p == 0.0 or r == 0.0:
        return 0.0
    return round(2.0 / (1.0/p + 1.0/r), 3)
def results_quality(true_list, predicted_list):
    p5 = precision_at_k(true_list, predicted_list, 5)
    print(p5)
    f1_30 = f1_at_k(true_list, predicted_list, 30)
    print(f1_30)
    if p5 == 0.0 or f1_30 == 0.0:
        return 0.0
    return round(2.0 / (1.0/p5 + 1.0/f1_30), 3)

# assert precision_at_k(range(10), [1,2,3] , 2) == 1.0
# assert recall_at_k(   range(10), [10,5,3], 2) == 0.1
# assert precision_at_k(range(10), []      , 2) == 0.0
# assert precision_at_k([],        [1,2,3],  5) == 0.0
# assert recall_at_k(   [],        [10,5,3], 2) == 1.0
# assert recall_at_k(   range(10), [],       2) == 0.0
# assert f1_at_k(       [],        [1,2,3],  5) == 0.0
# assert f1_at_k(       range(10), [],       2) == 0.0
# assert f1_at_k(       range(10), [0,1,2],  2) == 0.333
# assert f1_at_k(       range(50), range(5), 30) == 0.182
# assert f1_at_k(       range(50), range(10), 30) == 0.333
# assert f1_at_k(       range(50), range(30), 30) == 0.75
# assert results_quality(range(50), range(5))  == 0.308
# assert results_quality(range(50), range(10)) == 0.5
# assert results_quality(range(50), range(30)) == 0.857
# assert results_quality(range(50), [-1]*5 + list(range(5,30))) == 0.0


In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]
    print(tokens)
    return tokens

def get_synonyms(word):
    synonyms = []
    # if isinstance(word, tuple):
    #     filtered_string = ' '.join([str(item) for item in word if isinstance(item, str)])
    #     for syn in wordnet.synsets(filtered_string):
    #         for lemma in syn.lemmas():
    #             # Get the normalized form of the word from the lemma
    #             normalized_word = lemma.name()
    #             # Check if the word contains any signs
    #             if normalized_word.isalpha():
    #                 # Remove characters specified by the regular expression pattern
    #                 normalized_word = ' '.join(RE_WORD.findall(normalized_word))
    #                 normalized_word = normalized_word.lower()
    #                 if normalized_word:
    #                     synonyms.append(normalized_word)
    #     return synonyms
    # else:
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            # Get the normalized form of the word from the lemma
            normalized_word = lemma.name()
            # Check if the word contains any signs
            if normalized_word.isalpha():
                # Remove characters specified by the regular expression pattern
                # normalized_word = ' '.join(RE_WORD.findall(normalized_word))
                normalized_word = normalized_word.lower()
                if normalized_word:
                    synonyms.append(normalized_word)
    return synonyms

def find_most_similar(word, n=5):
    synonyms = get_synonyms(word)
    similarities = [(syn, wordnet.wup_similarity(wordnet.synsets(word)[0], wordnet.synsets(syn)[0]))
                    for syn in
                    synonyms if
                    syn != word and syn not in all_stopwords and wordnet.wup_similarity(wordnet.synsets(word)[0],
                                                                      wordnet.synsets(syn)[0]) is not None and
                    wordnet.wup_similarity(wordnet.synsets(word)[0], wordnet.synsets(syn)[0]) > 0.5]
    similarities.sort(key=lambda x: x[1], reverse=True)
    # Remove duplicates
    unique_similarities = []
    seen = set()
    for sim in similarities:
        if sim[0] not in seen:
            unique_similarities.append(sim)
            seen.add(sim[0])
    print(f'token : {word}, sim words: {unique_similarities}')
    return unique_similarities[:n]

query = "Who is considered the \"Father of the United States\"?"
query_tokens = tokenize(query)
print(query_tokens)
sim_query_tokens =[]
for token in query_tokens:
    sim_tokens = [t[0] for t in find_most_similar(token)]
    sim_query_tokens.extend(sim_tokens)
sim_query_tokens.extend(query_tokens)
print(sim_query_tokens)


['considered', 'father', 'united', 'states']
['considered', 'father', 'united', 'states']
token : considered, sim words: [('consider', 1.0)]
token : father, sim words: [('begetter', 1.0), ('mother', 0.9230769230769231), ('forefather', 0.782608695652174), ('beginner', 0.5454545454545454)]
token : united, sim words: [('unite', 1.0)]
token : states, sim words: [('state', 1.0), ('province', 1.0), ('commonwealth', 0.9411764705882353)]
['consider', 'begetter', 'mother', 'forefather', 'beginner', 'unite', 'state', 'province', 'commonwealth', 'considered', 'father', 'united', 'states']


In [15]:
# uncomment the code below and execute to reload the module when you make
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(sim_calc)
importlib.reload(se)
importlib.reload(inverted_index_title_gcp)
importlib.reload(inverted_index_body_gcp_training)

<module 'inverted_index_body_gcp_training' from '/content/inverted_index_body_gcp_training.py'>

In [24]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below.
url = "http://8a8b-34-106-45-248.ngrok-free.app"
qs_res = []
counter = 0
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
      res = requests.get(url + '/search', {'query': q}, timeout=35)
      duration = time() - t_start
      if res.status_code == 200:
        pred_wids, score = zip(*res.json())
        print(f'{pred_wids[0]} , score: {score[0]}, time: {duration}')
        rq = results_quality(true_wids, pred_wids)
        break
  except Exception as e:
    # Handle the exception and print it
    # print("An exception occurred:", e)
    pass

  qs_res.append((q, duration))
  print(f'q,time {qs_res} and {rq}')

17457578 , score: 1.0, time: 0.6930637359619141
0.0
0.0
